# Parameter Tuner

## Load

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob

from utils.calibrator import Calibrator
from utils.gradient_threshold import GradientThreshold
from utils.color_threshold import ColorThreshold
%matplotlib inline

test_image = cv2.imread("../media/test_images/test6.jpg")
test_imshape = test_image.shape

calibrator = Calibrator('../media/camera_cal/calibration*.jpg')
undist_image = calibrator.undistort(test_image)

grad_thresh = GradientThreshold(sobel_kernel=15, mag_thresh=(70, 255), dir_thresh=(0.5, 1.3))
color_thresh = ColorThreshold(white_s_thresh=(0, 255), white_b_thresh=(208, 255), yellow_s_thresh=(100, 255), yellow_b_thresh=(130, 255))


## 1. Distortion Correction Demo

In [2]:
images = glob.glob('../media/camera_cal/calibration*.jpg')
for fname in images:
    img = cv2.imread(fname)
    undist = calibrator.undistort(img)

    # Draw and display the corners
    cv2.imshow('undist',undist)
    cv2.waitKey(500)

cv2.destroyAllWindows()

## 2-1. Gradient Threshold Tuner
Threshold
- mag low: 50
- mag high: 255
- dir low: 0.7 rad
- dir high: 1.3 rad

In [17]:
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline


# update callback
def mod_grad_threshold(image, mag_low, mag_high, dir_low, dir_high):
    grad_thresh.set_mag_thresh((mag_low, mag_high))
    grad_thresh.set_dir_thresh((dir_low, dir_high))
    grad_binary = grad_thresh.grad_select(undist_image)

    plt.imshow(grad_binary, cmap='gray')


mag_low = widgets.IntSlider(min=0, max=255, description='mag low', value=50)
mag_high = widgets.IntSlider(min=0, max=255, description='mag high', value=255)
dir_low = widgets.FloatSlider(min=0, max=np.pi, description='mag low', value=0.7)
dir_high = widgets.FloatSlider(min=0, max=np.pi, description='mag high', value=1.3)

output = widgets.interact(lambda mag_low, mag_high, dir_low, dir_high: \
                mod_grad_threshold(undist_image, mag_low, mag_high, dir_low, dir_high),
                mag_low=mag_low,
                mag_high=mag_high,
                dir_low=dir_low,
                dir_high=dir_high)

interactive(children=(IntSlider(value=50, description='mag low', max=255), IntSlider(value=255, description='m…

## 2-2. Color Tuner
**White**
- Saturation (0, 255)
- Brightness (208, 255)

**Yellow**
- Saturation (100, 255)
- Brightness (130, 255)

In [20]:
import ipywidgets as widgets
from IPython.display import display

# update color threshold
def mod_hls_threshold(image, s_thresh_low, s_thresh_high, b_thresh_low, b_thresh_high):
    color_thresh.set_white_thresh((s_thresh_low, s_thresh_high), (b_thresh_low, b_thresh_high))

    hls_binary = color_thresh.hls_select_white(image)
    plt.imshow(hls_binary, cmap='gray')

s_thresh_low = widgets.IntSlider(min=0, max=255, description='sat low', value=150)
s_thresh_high = widgets.IntSlider(min=0, max=255, description='sat high', value=255)
b_thresh_low = widgets.IntSlider(min=0, max=255, description='b low', value=200)
b_thresh_high = widgets.IntSlider(min=0, max=255, description='b high', value=255)

output = widgets.interact(lambda s_thresh_low, s_thresh_high, b_thresh_low, b_thresh_high: \
                mod_hls_threshold(undist_image, s_thresh_low, s_thresh_high, b_thresh_low, b_thresh_high),
                s_thresh_low=s_thresh_low,
                s_thresh_high=s_thresh_high,
                b_thresh_low=b_thresh_low, 
                b_thresh_high=b_thresh_high)

interactive(children=(IntSlider(value=150, description='sat low', max=255), IntSlider(value=255, description='…

## 3-1. Finding Region of Interest
Corner Point
- 575, 460
- 705, 460
- 1130, 720
- 150, 720

In [3]:
import ipywidgets as widgets
from IPython.display import display
%matplotlib qt

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def mod_region(image, x1, y1, x2, y2):
    imshape = image.shape
    region_image = region_of_interest(image, np.array([[(150, imshape[0]),(x1, y1), (x2, y2), (1130,imshape[0])]]))

    cv2.imshow("region", region_image)

test_image = cv2.imread("../media/test_images/straight_lines1.jpg")
test_imshape = test_image.shape
undist_image = calibrator.undistort(test_image)

print(test_imshape)
x1_w = widgets.IntSlider(min=0, max=test_imshape[1], description='X1', value=575)
y1_w = widgets.IntSlider(min=0, max=test_imshape[0], description='Y1', value=460)
x2_w = widgets.IntSlider(min=0, max=test_imshape[1], description='X1', value=705)
y2_w = widgets.IntSlider(min=0, max=test_imshape[0], description='Y1', value=460)

output = widgets.interact(lambda x1, y1, x2, y2:
                mod_region(undist_image, x1, y1, x2, y2),
                x1=x1_w, y1=y1_w, x2=x2_w, y2=y2_w)

(720, 1280, 3)


interactive(children=(IntSlider(value=575, description='X1', max=1280), IntSlider(value=460, description='Y1',…